In [ ]:
import catboost as cb
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import StratifiedKFold,KFold
import joblib
import os
from sklearn.metrics import f1_score
import datetime
pd.set_option('display.max_columns', 100) # 设置显示最大列数

In [ ]:
data = pd.read_csv('../data/data_4feature_all.csv')

In [ ]:
coloums_list = joblib.load('../data/coloums_list')

In [ ]:
cut_data = data[coloums_list]

In [ ]:
feas=[i  for i in cut_data.columns.tolist() if i not in ['id','label']]

x_train = cut_data[feas]
y_train = cut_data['label']
folds = 5
seed = 2023
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

cv_scores = []
test_pre = []
Feass = pd.DataFrame()
train = np.zeros(x_train.shape[0])

current_time = datetime.datetime.now()
current_time = str(current_time).replace(' ','-')
os.mkdir('../log/'+current_time)
models = []

params={
    'loss_function': 'Logloss', # 损失函数，取值RMSE, Logloss, MAE, CrossEntropy, Quantile, LogLinQuantile, Multiclass, MultiClassOneVsAll, MAPE, Poisson。默认Logloss。
    'custom_loss': 'F1', # 训练过程中计算显示的损失函数，取值Logloss、CrossEntropy、Precision、Recall、F、F1、BalancedAccuracy、AUC等等
    'eval_metric': 'F1', # 用于过度拟合检测和最佳模型选择的指标，取值范围同custom_loss
    'iterations': 500, # 最大迭代次数，默认500. 别名：num_boost_round, n_estimators, num_trees
    'learning_rate': 0.15, # 学习速率,默认0.03 别名：eta
    'random_seed': 2023, # 训练的随机种子，别名：random_state
    'subsample': 0.9, # 样本采样比率（行采样）
    'use_best_model': True, # 让模型使用效果最优的子树棵树/迭代次数，使用验证集的最优效果对应的迭代次数（eval_metric：评估指标，eval_set：验证集数据），布尔类型可取值0，1（取1时要求设置验证集数据）
    'depth': 7, # 树深，默认值6
    'nan_mode': 'Max', # 缺失值处理方法，取值Forbidden（不支持缺失值，输入包含缺失时会报错）、Min（处理为该列的最小值，比原最小值更小）、Max（同理）
    # 'max_leaves': 12, # 最大叶子结点数量
    'rsm': 0.9, # 列采样比率，别名colsample_bylevel 取值（0，1],默认值1
    # 'nan_mode': 'Max', # 缺失值处理方法，取值Forbidden（不支持缺失值，输入包含缺失时会报错）、Min（处理为该列的最小值，比原最小值更小）、Max（同理）
    'logging_level':'Silent', # 模型训练过程的信息输出等级，取值Silent（不输出信息）、Verbose（默认值，输出评估指标、已训练时间、剩余时间等）、Info（输出额外信息、树的棵树）、Debug（debug信息）
    'early_stopping_rounds': 50,
}
for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):
    print('************************************ {} ************************************'.format(str(i+1)))
    trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]
    cbc = cb.CatBoostClassifier(**params)
    cbc.fit(trn_x, trn_y,eval_set=(val_x, val_y))
    models.append(cbc)
    joblib.dump(cbc, f"../log/{current_time}/cbm_model{i+1}.pkl")
    proba = cbc.predict_proba(val_x)
    val_pred = np.where(proba[:,0]>0.85,0,1)
    train[valid_index] = val_pred
    print(f1_score(val_pred,val_y))
    # print(f1_score(cbc.predict(val_x),val_y))
f1 = f1_score(y_train,train)
print(f1)
os.rename(f'../log/{current_time}',f'../log/{f1}')